In [1]:
import os
from IPMOFresults import *

# Folder containing results file
#outputDir = "C:\\Kutay\\IPMOF_Output"
outputDir = r'C:\Kutay\IPMOF_MOF5\lowE2'

In [7]:
baseMOF = os.listdir(outputDir)
table = []
totalStructureCount = 0
completed = '---'
simCount = 0

for bMOF in baseMOF:
    mobileMOF = os.listdir(os.path.join(outputDir,bMOF))
    
    for mMOF in mobileMOF:
        simPar = os.listdir(os.path.join(outputDir,bMOF, mMOF))
        
        for sim in simPar:
            simDir = os.path.join(outputDir, bMOF, mMOF, sim)
            resultsDir = os.path.join(outputDir,bMOF, mMOF, sim, 'results.txt')
            
            # Check if job .out file exists
            for simFile in os.listdir(simDir):
                if '.out' in simFile:
                    jobFileDir = os.path.join(outputDir,bMOF, mMOF, sim, simFile)
                    runTime = readJobFile(jobFileDir)
                    #runTime = '01:00'
                    break
                else:
                    runTime = '---'
            
            # Check if results file exists
            if os.path.isfile(resultsDir):
                resultsSize = os.path.getsize(resultsDir)
                
                if resultsSize > 0:
                    summary = readSummary(simDir)
                    IPstructures = readStructures(simDir)
                    minEstructures = getMinEnergyStructures(IPstructures, 1)
                    if len(summary['percent']) > 0:
                        completed = summary['percent'][-1]
                        totalStructureCount = summary['structureCount'][-1]
                    else:
                        completed = 'Interrupted'
                        totalStructureCount = 'Interrupted'
                        totalStructureCount = IPstructures['structureIndex'][-1]
                else:
                    completed = 'Error'
                    totalStructureCount = 'Err'
                    
            else:
                completed = 'NoResults'
                totalStructureCount = 'NoResults'
                           
        table.append([])
        table[simCount].append(bMOF)
        table[simCount].append(mMOF)
        table[simCount].append(sim)
        table[simCount].append(completed)
        table[simCount].append(runTime)
        table[simCount].append(totalStructureCount)
        simCount += 1

completeSIM = []
errorSIM = []
interruptedSIM = []
noresSIM = []
for refSim in table:
    if refSim[3] == 100:
        completeSIM.append(refSim)
    if refSim[3] == 'Error':
        errorSIM.append(refSim)
    if refSim[3] == 'Interrupted':
        interruptedSIM.append(refSim)
    if refSim[3] == 'NoResults':
        noresSIM.append(refSim)

del table

from tabulate import tabulate

headers = ["Base MOF", "Mobile MOF", "SimPar", "Completed", "RunTime", "sCount"]
print(tabulate(completeSIM, headers))

Base MOF    Mobile MOF    SimPar      Completed    RunTime    sCount
----------  ------------  --------  -----------  ---------  --------
MOF5        MOF5          simulPar          100        151         6


In [10]:
simTime = []
for comSim in completeSIM:
    if comSim[4] > 0:
        simTime.append(comSim[4])
    else:
        # If simulation started before end of day and finished after midnight
        simTime.append(comSim[4]+24*3600)
        
# Index of simulation with maximum time
maxTimeIndex = simTime.index(sorted(simTime)[-1])
minTimeIndex = simTime.index(sorted(simTime)[0])
#print(completeSIM[maxTimeIndex])

# Convert time from seconds to string format
import datetime
print('Maximum time: ' + str(datetime.timedelta(seconds=maxTimeIndex)))
print('Minimum time: ' + str(datetime.timedelta(seconds=minTimeIndex)))

# Average Time
avgTime = sum(simTime)/len(simTime)
print('Average Time: ' + str(datetime.timedelta(seconds=avgTime)))

Maximum time: 0:00:00
Minimum time: 0:00:00
Average Time: 0:02:31


In [11]:
sCount = []
sIndex = []
for comIndex, comSim in enumerate(completeSIM):
    if comSim[5] > 0:
        sCount.append(comSim[5])
        sIndex.append(comIndex)
        
# Index of simulation with maximum time
maxSindex = sIndex[sCount.index((sorted(sCount)[-1]))]
maxScount = completeSIM[maxSindex]
minSindex = sIndex[sCount.index((sorted(sCount)[0]))]
minScount = completeSIM[minSindex]

# Convert time from seconds to string format
print('Maximum Structure: ' + str(maxScount))
print('Minimum Structure: ' + str(minScount))

# Average Time
avgStructure = sum(sCount)/len(sCount)
print('Average Structure Count: ' + str(avgStructure))
print('IPs with zero structures: ' +str(len(completeSIM)-len(sCount)))
print('IPs with non-zero structures: ' +str(len(sCount)))

Maximum Structure: ['MOF5', 'MOF5', 'simulPar', 100.0, 151, 6.0]
Minimum Structure: ['MOF5', 'MOF5', 'simulPar', 100.0, 151, 6.0]
Average Structure Count: 6.0
IPs with zero structures: 0
IPs with non-zero structures: 1


In [5]:
sCount = []
sIndex = []
for comIndex, comSim in enumerate(completeSIM):
    sCount.append(comSim[5])
    sIndex.append(comIndex)
        
# Index of simulation with maximum time
maxSindex = sIndex[sCount.index((sorted(sCount)[-1]))]
maxScount = completeSIM[maxSindex]

#print(completeSIM[maxTimeIndex])

# Convert time from seconds to string format
print('Maximum Structure: ' + str(maxScount))
#print('Minimum time: ' + str(datetime.timedelta(seconds=minTimeIndex)))

# Average Time
avgStructure = sum(sCount)/len(sCount)
print('Average Structure Count: ' + str(avgStructure))

Maximum Structure: ['VUSJUP', 'VUSJUP', 'E-2_F12_L50_S', 100.0, -85461, 68027.0]
Average Structure Count: 1078.981549815498


In [14]:
import datetime

for comSimIndex, comSim in enumerate(completeSIM):
    if comSim[4] < 0:
        comSim[4] = comSim[4]+24*3600

homoIP = []
heteroIP = []
for comIndex, comSim in enumerate(completeSIM):
    if comSim[0] == comSim[1]:
        homoIP.append(comSim)
    else:
        heteroIP.append(comSim)

homoTime = []
homoStructure = []
homoZero = 0
homoNonZero = 0
homoTotal = 0
for homo in homoIP:
    homoTime.append(homo[4])
    homoStructure.append(homo[5])
    if homo[5] == 0:
        homoZero += 1
    else: 
        homoNonZero += 1
    homoTotal += 1
    
heteroTime = []
heteroStructure = []
heteroZero = 0
heteroNonZero = 0
heteroTotal = 0
for hetero in heteroIP:
    heteroTime.append(hetero[4])
    heteroStructure.append(hetero[5])
    if hetero[5] == 0:
        heteroZero += 1
    else: 
        heteroNonZero += 1
    heteroTotal += 1
    

homoAvgTime = sum(homoTime)/len(homoTime)
print('HomoIP Average Time: ' + str(datetime.timedelta(seconds=homoAvgTime)))
heteroAvgTime = sum(heteroTime)/len(heteroTime)
print('Hetero IP Average Time: ' + str(datetime.timedelta(seconds=heteroAvgTime)))

homoAvgStructure = sum(homoStructure)/len(homoStructure)
print('HomoIP Average Structure: ' + str(homoAvgStructure))
heteroAvgStructure = sum(heteroStructure)/len(heteroStructure)
print('Hetero IP Average Structure: ' + str(heteroAvgStructure))

# Convert time from seconds to string format
print('HomoIP Total Structure: ' + str(homoTotal))
print('HomoIP Zero Structure: ' + str(homoZero))
print('HomoIP NonZero Structure: ' + str(homoNonZero))

# Convert time from seconds to string format
print('HeteroIP Total Structure: ' + str(heteroTotal))
print('HeteroIP Zero Structure: ' + str(heteroZero))
print('HeteroIP NonZero Structure: ' + str(heteroNonZero))

HomoIP Average Time: 0:04:08.812500
Hetero IP Average Time: 0:03:13.215686
HomoIP Average Structure: 4986.375
Hetero IP Average Structure: 833.8117647058823
HomoIP Total Structure: 16
HomoIP Zero Structure: 6
HomoIP NonZero Structure: 10
HeteroIP Total Structure: 255
HeteroIP Zero Structure: 130
HeteroIP NonZero Structure: 125


In [11]:
homoTime

[290,
 143,
 167,
 52,
 77,
 100,
 52,
 289,
 461,
 140,
 517,
 -85461,
 -85923,
 69,
 157,
 51]

In [29]:
# Only simulations where IP structures were found (135)
simTime = []
for comSim in completeSIM:
    if comSim[5] > 0:
        if comSim[4] > 0:
            simTime.append(comSim[4])
        else:
            simTime.append(comSim[4]+24*3600)

# Minimum Time            
str(datetime.timedelta(seconds=sorted(simTime)[0]))

# Avg Time
avgTime = sum(simTime)/len(simTime)
print('Average Time: ' + str(datetime.timedelta(seconds=avgTime)))

Average Time: 0:04:46.177778


In [22]:
outputDir = r'C:\Kutay\IPMOF_MOF5\lowE2\MOF5\MOF5\simulPar'
resultsFileName = 'results.txt'
resDir = os.path.join(outputDir, resultsFileName)
resFile = open(resDir, 'r')
resLines = resFile.readlines()
resFile.close()

In [48]:

structureCount = 0

for line in resLines:
    if 'Base' in line:
        break
        
    if '---' in line:
        structureCount += 1
        lineIndex = resLines.index(line) + 1
        nextLine = resLines[lineIndex]
        if '---' in nextLine:
            structureCount -= 1
        if 'Base' in nextLine:
            structureCount -= 1


In [10]:
IPstructures['xyz'][0]

['O 6.458 6.458 6.45',
 'O 7.282 5.634 3.46',
 'C 6.458 6.458 2.87',
 'C 6.458 6.458 1.3',
 'C 7.308 5.608 0.69',
 'H 7.876 5.04 1.15',
 'Zn 18.253 20.495 5.33',
 'O 19.374 19.374 6.45',
 'O 18.55 20.198 3.46',
 'C 19.374 19.374 2.87',
 'C 19.374 19.374 1.3',
 'C 18.524 20.224 0.69',
 'H 17.956 20.792 1.15',
 'Zn 18.253 5.337 20.49',
 'O 19.374 6.458 19.37',
 'O 18.55 5.634 22.37',
 'C 19.374 6.458 22.95',
 'C 19.374 6.458 24.44',
 'C 18.524 5.608 25.13',
 'H 17.956 5.04 24.67',
 'Zn 7.579 20.495 20.49',
 'O 6.458 19.374 19.37',
 'O 7.282 20.198 22.37',
 'C 6.458 19.374 22.95',
 'C 6.458 19.374 24.44',
 'C 7.308 20.224 25.13',
 'H 7.876 20.792 24.67',
 'Zn 5.337 7.579 5.33',
 'O 3.461 7.282 5.63',
 'C 2.875 6.458 6.45',
 'C 1.39 6.458 6.45',
 'C 0.695 7.308 5.60',
 'H 1.157 7.876 5.0',
 'Zn 5.337 18.253 20.49',
 'O 3.461 18.55 20.19',
 'C 2.875 19.374 19.37',
 'C 1.39 19.374 19.37',
 'C 0.695 18.524 20.22',
 'H 1.157 17.956 20.79',
 'Zn 20.495 18.253 5.33',
 'O 22.371 18.55 5.63',
 'C 

In [13]:
IPstructures['xyz'][0][0]

'O 6.458 6.458 6.45'

In [19]:
exportDir = r'C:\Kutay\IPMOF_MOF5\lowE2\MOF5\MOF5\simulPar'
xyzFileName = 'MOF5IP'
IPindex = 0


In [20]:
exportIPxyz(0, xyzFileName, exportDir)

In [2]:
sourceDir = r'C:\Kutay\poreblazer_v3.0.2_d\Source'

In [5]:
os.listdir(sourceDir)

['defaults.dat', 'poreblazer.exe', 'UFF.atoms']

In [7]:
import shutil

In [9]:
shutil.copy?